# Kaggle : Airbnb New User Bookings

In [2]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
import seaborn as sns

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [39]:
import warnings
warnings.filterwarnings("ignore")

# data

# sessions

In [ ]:
#10M data
sessions = pd.read_csv("data/sessions.csv")
sessions.head()

In [ ]:
sessions.info()

In [ ]:
sessions.isnull().sum()

In [ ]:
#359 action type
sessions['action'].value_counts()

In [ ]:
sessions['action_type'].value_counts()

In [ ]:
#155 action detail
sessions['action_detail'].value_counts()

In [ ]:
sessions['secs_elapsed'].describe()

# age_gender_bkts

In [ ]:
#21 age bucket (5 years) per gender by country
#1K unit
age_gender = pd.read_csv("data/age_gender_bkts.csv")
age_gender.head()

In [ ]:
age_gender.info()

# countries (destination)

- AU = austrailia
- CA = canada
- DE = germany
- ES = spain
- FR = france
- GB = england
- IT = italia
- NL = netherland 
- PT = Portugal
- US = USA

In [ ]:
countries = pd.read_csv("data/countries.csv")
countries

In [ ]:
countries['country_destination'].value_counts()

# submission

In [ ]:
# submission = pd.read_csv("data/sample_submission_NDF.csv")
# submission.head()

In [ ]:
# submission.info()

# test

In [ ]:
test = pd.read_csv("data/test_users.csv")
test.head()

In [ ]:
test.isnull().sum()

# train

In [ ]:
train = pd.read_csv("data/train_users_2.csv")
train.head()

In [ ]:
train.isnull().sum()

# drop date_first_booking column

In [ ]:
train = train.drop(labels=['date_first_booking'], axis=1)
test = test.drop(labels=['date_first_booking'], axis=1)

# first_affiliate_tracked nan data manipulation

In [ ]:
train['first_affiliate_tracked'] = train.first_affiliate_tracked.replace(np.nan, "untracked")
test['first_affiliate_tracked'] = test.first_affiliate_tracked.replace(np.nan, "untracked")

# age

In [ ]:
print(len(train[train['age'] > 120]))

In [ ]:
train[train['age'] > 120]

In [ ]:
train_copy = train.copy()
train_copy = train_copy[train_copy['age'] <= 120]
train_copy.isnull().sum()

In [ ]:
train_copy = train.copy()
train_copy = train_copy.dropna()
train_copy = train_copy[train_copy['age'] <= 120]

ax1 = sns.distplot(train_copy['age'])
plt.show()
ax2 = sns.boxplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
plt.show()
# ax3 = sns.swarmplot(x = train_copy['country_destination'], y = train_copy['age'], data = train_copy)
# plt.show()

# age nan value manipulation

- DBSCAN
- heatmap age vs related features

# pre processing

In [10]:
import pandas as pd

In [11]:
df_test_id = pd.read_csv("data/test_users.csv")

In [12]:
df_test_id = df_test_id['id']

In [13]:
df_sample = pd.read_csv('data/sample_submission_NDF.csv')

In [14]:
df_train = pd.read_csv("data/train_users_2.csv")
df_test = pd.read_csv("data/test_users.csv")
df_all = pd.concat([df_train, df_test])

In [15]:
df_target = df_train["country_destination"]
df_values = df_train["country_destination"].values

In [16]:
df_all = df_all.drop("age", axis = 1)
df_all = df_all.drop("date_first_booking", axis = 1)

In [17]:
df_all["first_affiliate_tracked"].fillna("untracked", inplace = True)

In [18]:
df_all.isnull().sum()

affiliate_channel              0
affiliate_provider             0
country_destination        62096
date_account_created           0
first_affiliate_tracked        0
first_browser                  0
first_device_type              0
gender                         0
id                             0
language                       0
signup_app                     0
signup_flow                    0
signup_method                  0
timestamp_first_active         0
dtype: int64

In [19]:
df_all["date_account_created"] = pd.to_datetime(df_all["date_account_created"], format = "%Y-%m-%d")

In [20]:
df_all["timestamp_first_active"] = pd.to_datetime(df_all["timestamp_first_active"], format="%Y%m%d%H%M%S", errors='ignore')

In [21]:
df_all["ac_year"] = df_all["date_account_created"].apply(lambda x : x.year)
df_all["ac_month"] = df_all["date_account_created"].apply(lambda x : x.month)
df_all["ac_day"] = df_all["date_account_created"].apply(lambda x : x.day)

In [22]:
df_all["fa_year"] = df_all["timestamp_first_active"].apply(lambda x : x.year)
df_all["fa_month"] = df_all["timestamp_first_active"].apply(lambda x : x.month)
df_all["fa_day"] = df_all["timestamp_first_active"].apply(lambda x : x.day)
df_all["fa_hour"] = df_all["timestamp_first_active"].apply(lambda x : x.hour)
df_all["fa_minute"] = df_all["timestamp_first_active"].apply(lambda x : x.minute)
df_all["fa_second"] = df_all["timestamp_first_active"].apply(lambda x : x.second)

In [23]:
df_all = df_all.drop("date_account_created", axis = 1)
df_all = df_all.drop("timestamp_first_active", axis = 1)
df_all = df_all.drop("country_destination", axis = 1)

In [24]:
df_all = df_all.drop("id", axis = 1)

In [25]:
df_all = pd.get_dummies(df_all)

In [26]:
df_train = df_all.iloc[:213451]
df_test = df_all.iloc[213451:]

In [27]:
df_train.shape, df_test.shape, df_target.shape

((213451, 145), (62096, 145), (213451,))

In [28]:
df_target.value_counts()

NDF      124543
US        62376
other     10094
FR         5023
IT         2835
GB         2324
ES         2249
CA         1428
DE         1061
NL          762
AU          539
PT          217
Name: country_destination, dtype: int64

In [29]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_label = le.fit_transform(df_target) 

In [30]:
y_label

array([ 7,  7, 10, ...,  7,  7,  7])

# Logistic Regression

In [31]:
from sklearn import linear_model

In [32]:
logreg = linear_model.LogisticRegression(n_jobs=4, random_state=0)

# we create an instance of Neighbours Classifier and fit the data.
model_log = logreg.fit(df_train, y_label)

/home/jk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


In [37]:
# y_pred_log = model_log.predict(df_test)
# y_inverse_log = le.inverse_transform(y_pred_log)
# y_inverse_log
# df_sample['country'] = y_inverse_log
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_log.csv', sep=',', na_rep='NaN', index = False)
#0.69443 #1200/1460

In [40]:
#predict_proba
y_log1 = model_log.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_log1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_log1.csv', sep=',', na_rep='NaN', index = False)
#0.85906
df_sample1.country.value_counts()

other    62096
US       62096
NDF      62096
FR       62096
IT       47471
ES        8161
GB        5900
DE         445
CA         119
Name: country, dtype: int64

# QDA / LDA

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
qda = QuadraticDiscriminantAnalysis().fit(df_train, y_label)

In [ ]:
#predict_proba
y_qda1 = qda.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_qda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_qda1.csv', sep=',', na_rep='NaN', index = False)


df_sample1.country.value_counts()

In [41]:
# y_qda = qda.predict(df_test)
# y_inverse_qda = le.inverse_transform(y_qda)
# y_inverse_qda

# df_sample['country'] = y_inverse_qda
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_qda.csv', sep=',', na_rep='NaN', index = False)
# #0.00289 #1300/1460

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis().fit(df_train, y_label)

In [ ]:
#predict_proba
y_lda1 = lda.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lda1.csv', sep=',', na_rep='NaN', index = False)
#0.85654
df_sample1.country.value_counts()

In [42]:
# y_lda = lda.predict(df_test)
# y_inverse_lda = le.inverse_transform(y_lda)
# y_inverse_lda

# df_sample['country'] = y_inverse_lda
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_lda.csv', sep=',', na_rep='NaN', index = False)
# #0.68756 #1210/1460

# Naive Bayesian

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model_nb = MultinomialNB().fit(df_train, y_label)

In [ ]:
#predict_proba
y_nb1 = model_nb.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lda1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_nb1.csv', sep=',', na_rep='NaN', index = False)
#0.85654
df_sample1.country.value_counts()

In [ ]:
# y_nb = model_nb.predict(df_test)
# y_inverse_nb = le.inverse_transform(y_nb)
# y_inverse_nb

# df_sample['country'] = y_inverse_nb
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_nb.csv', sep=',', na_rep='NaN', index = False)
# #0.54326 #1260/1460

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#max_depth = 10 is the best score
tree1 = DecisionTreeClassifier(max_depth=10).fit(df_train, y_label)

In [ ]:
#predict_proba
y_tree1 = tree1.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_tree1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_tree1.csv', sep=',', na_rep='NaN', index = False)
#0.85856
df_sample1.country.value_counts()

In [ ]:
# y_tree = tree1.predict(df_test)
# y_inverse_tree = le.inverse_transform(y_tree)
# y_inverse_tree

# df_sample['country'] = y_inverse_tree
# df_sample = df_sample.reset_index(drop=True)
# df_sample.to_csv('sub_tree.csv', sep=',', na_rep='NaN', index = False)
# #0.69341 #1200/1460

# randomforest

In [ ]:
from sklearn import clone
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_random = RandomForestClassifier(max_depth=10, random_state=0).fit(df_train, y_label)

In [ ]:
#predict_proba
y_random1 = model_random.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_random1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_random1.csv', sep=',', na_rep='NaN', index = False)
#0.85346

In [ ]:
y_random = model_random.predict(df_test)
y_inverse_random = le.inverse_transform(y_random)
y_inverse_random

In [ ]:
df_sample['country'] = y_inverse_random
df_sample = df_sample.reset_index(drop=True)
df_sample.to_csv('sub_random.csv', sep=',', na_rep='NaN', index = False)
#0.67929 #1250/1460

In [ ]:
df_sample.country.value_counts()

# extratree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
model_extra = ExtraTreesClassifier(max_depth=4, random_state=0).fit(df_train, y_label)

In [ ]:
#predict_proba
y_extra1 = model_extra.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_extra1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_extra1.csv', sep=',', na_rep='NaN', index = False)
#0.85359

# support vector machine

In [55]:
from sklearn import svm

In [ ]:
#linear
model_svm1 = svm.LinearSVC(random_state=0).fit(df_train, y_label)

In [26]:
#predict_proba
y_svm1 = model_svm1.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_svm1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_svm1.csv', sep=',', na_rep='NaN', index = False)
#0.85359

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [ ]:
#kernel
model_svm = svm.SVC(random_state=0, probability=True).fit(df_train, y_label)

In [ ]:
#predict_proba
y_svm = model_svm.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_svm[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_svm.csv', sep=',', na_rep='NaN', index = False)
#0.85359

# age column

In [ ]:
test_age = pd.read_csv("last_test_data.csv")
train_age = pd.read_csv("last_train_data.csv")

test_age = test_age['age']
train_age = train_age['age']

In [ ]:
train_age.head(), len(train_age)

In [ ]:
test_age.head(), len(test_age)

In [ ]:
df_train = pd.concat([df_train, train_age], axis=1)
df_test = pd.concat([df_test, test_age], axis=1)

In [ ]:
df_train.info(), df_test.info(), 

# age clustering

In [ ]:
df_all = pd.concat([df_train, df_test], axis=0)
df_all.head()

In [ ]:
def pre_age_predict_data_cat():
    
    bins = [0, 15, 25, 35, 60, 9999]
    labels = ["미성년자", "청년", "중년", "장년", "노년"]
    cats = pd.cut(df_all['age'], bins, labels=labels)
    cats = pd.DataFrame(cats)
    
    return cats
cats=pre_age_predict_data_cat()

In [ ]:
cats_d = pd.get_dummies(cats)

In [ ]:
len(cats_d)

In [ ]:
train_cat = cats_d[:213451]
test_cat = cats_d[213451:]

In [ ]:
# df_train = df_train.drop("age", axis=1)
# df_test = df_test.drop("age", axis=1)

In [ ]:
df_train = pd.concat([df_train, train_cat], axis=1)
df_test = pd.concat([df_test, test_cat], axis=1)

In [ ]:
df_train.to_csv('df_train.csv', sep=',', na_rep='NaN', index = False)
df_test.to_csv('df_test.csv', sep=',', na_rep='NaN', index = False)

# XGBoost

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier

In [ ]:
model_xgb = xgb.XGBClassifier(max_depth=4 ,objective= 'multi:softmax', n_jobs=4 ).fit(df_train, y_label)

In [ ]:
#predict_proba
y_xgb1 = model_xgb.predict_proba(df_test)

ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_xgb1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_xgb1.csv', sep=',', na_rep='NaN', index = False)
#0.85950 w/o age
#0.86442 with age
#0.85983 w/ dummy age

In [ ]:
dtrain = xgb.DMatrix(df_train, label = y_label)

In [ ]:
y_xgb = model_xgb.predict(df_test)
y_inverse_xgb = le.inverse_transform(y_xgb)
y_inverse_xgb

In [ ]:
df_sample['country'] = y_inverse_random
df_sample = df_sample.reset_index(drop=True)
df_sample.to_csv('sub_xgb.csv', sep=',', na_rep='NaN', index = False)
#0.67929 #1250/1460

In [ ]:
df_sample.country.value_counts()

# LightGBM

In [45]:
import lightgbm as lgb

In [46]:
model_lgb = lgb.LGBMClassifier(max_depth=10, objective = 'multiclass', random_state=0, n_jobs=4).fit(df_train, y_label)

In [47]:
y_vali = model_lgb.predict(df_train)

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report

In [50]:
print(classification_report(y_label, y_vali))

             precision    recall  f1-score   support

          0       1.00      0.00      0.01       539
          1       1.00      0.00      0.00      1428
          2       1.00      0.01      0.01      1061
          3       1.00      0.00      0.00      2249
          4       1.00      0.00      0.00      5023
          5       0.00      0.00      0.00      2324
          6       1.00      0.00      0.00      2835
          7       0.66      0.87      0.75    124543
          8       1.00      0.00      0.01       762
          9       1.00      0.00      0.01       217
         10       0.48      0.36      0.41     62376
         11       1.00      0.00      0.00     10094

avg / total       0.64      0.62      0.56    213451



In [51]:
#predict_proba
y_lgb = model_lgb.predict_proba(df_test)



ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lgb[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lgb.csv', sep=',', na_rep='NaN', index = False)
#0.85932 w/o age
#0.86495 w/ age

In [54]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model_lgb, df_train, y_label, cv=5, scoring='neg_log_loss')
scores

4

# light train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtr, Xv, ytr, yv = train_test_split(df_train, y_label, test_size=0.2, random_state=0)
dtrain = lgb.Dataset(Xtr, label=ytr)
dvalid = lgb.Dataset(Xv, label=yv)
# dtest = lgb.Dataset(df_test.values)

In [ ]:
params = {}

params['application'] = 'lambdarank'
params['max_depth'] = 8
params['metric'] = 'ndcg'
# params['metric'] = 'multi_error'
# params['boosting'] = 'rf'

In [ ]:
params

In [ ]:
model_lgb1 = lgb.train(params, dtrain, valid_sets=dvalid)

In [ ]:
model_lgb1

In [ ]:
len(y_lgb1)

In [ ]:
y_lgb1 = model_lgb1.predict(df_test)



ids = []  #list of ids
cts = []  #list of countries
for i in range(len(df_sample)):
    idx = df_sample.iloc[i, 0]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_lgb1[i])[::-1])[:5].tolist()

df_sample1 = pd.DataFrame(columns=['id', 'country'])
df_sample1['id'] = ids
df_sample1['country'] = cts
df_sample1 = df_sample1.reset_index(drop=True)
df_sample1.to_csv('sub_lgb1.csv', sep=',', na_rep='NaN', index = False)


# Cross validation

### k fold stacking

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(ntrian, n_folds=nfolds, shuffle = kfodl_shuffle, random_state = kfold_random_state)

oof_train = np.zeros((ntrain,))
oof_test = np.zeros((ntest,))
oof_test_skf = np.empty((nfolds, ntest))

In [ ]:
for i, (train_index, cross_index) in enumerate(kf):
    x_tr, x_cr = x_train.iloc[train_index], x_train.iloc[cross_index]
    y_tr, y_cr = y_train.iloc[train_index], y_train.iloc[cross_index]
    
    clf.train(x_tr, y_tr, x_cr, y_cr)

In [ ]:
oof_train[cross_index] = clf.predict(x_cr)
oof_test_skf[i, :] = clf.predict(x_test)
oof_train[:] = oof_test_skf.mean(axis=0)


In [ ]:
import numpy as np

from sklearn.model_selection import cross_val_predict, KFold
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

X = np.array([[1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4], [1, 2], [2, 4], [3, 2], [4, 4], [5, 2], [6, 4], [7, 2], [8, 4]])
Y = np.array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8])

Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,train_size=0.5,random_state=1)

clf = LinearDiscriminantAnalysis()
clf.fit(X, Y)

# without cross-valdidation
prediction = clf.predict(Xtest)

# with cross-valdidation
cv = KFold(n_splits=2)
prediction_cv = cross_val_predict(clf, X, Y, cv=cv)
prediction_cv